In [1]:
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [3]:
raw_data.head()

,laser0,laser1,laser2,laser3,laser4,laser5,laser6,laser7,laser8,laser9,...,laser352,laser353,laser354,laser355,laser356,laser357,laser358,laser359,speed,steering_angle
0,0.228333,0.227496,0.234184,0.253645,0.238905,0.224904,0.219742,0.225911,0.236172,0.221523,...,0.224054,0.248701,0.230076,0.243954,0.245454,0.225626,0.246615,0.244948,0.244948,0.585937
1,0.228333,0.227496,0.234184,0.253645,0.238905,0.224904,0.219742,0.225911,0.236172,0.221523,...,0.224054,0.248701,0.230076,0.243954,0.245454,0.225626,0.246615,0.244948,0.244948,0.585937
2,0.231983,0.244641,0.244981,0.224927,0.235483,0.222170,0.225249,0.235063,0.217606,0.233925,...,0.233786,0.242592,0.240255,0.225518,0.239098,0.246102,0.236991,0.250182,0.250182,0.600930
3,0.237585,0.226259,0.225327,0.241955,0.234837,0.225581,0.239836,0.223831,0.220821,0.220462,...,0.239585,0.228006,0.226670,0.248297,0.234633,0.228943,0.237144,0.233428,0.233428,0.600727
4,0.222417,0.230397,0.242593,0.251118,0.238735,0.237531,0.239533,0.239257,0.240184,0.254118,...,0.244356,0.262002,0.238537,0.229623,0.220751,0.249123,0.232163,0.234454,0.234454,0.591369


In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Concatenate, Input
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

In [ ]:
data_start = 0
data_stop = 1000
data_part = raw_data[data_start:data_stop]
speed = data_part["speed"].values
steering_angle = data_part["steering_angle"].values
output = np.dstack((steering_angle, speed))[0]

img_input = []
h = 240
w = 320
c = 3
for i in range(data_start+1, data_stop+1):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
    img = cv2.resize(img, (w, h))
    img_input.append(img)
img_input = np.asarray(img_input).astype(np.float32) / 255.0
print("img input shape:",img_input.shape)

X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)
if len(X_train.shape) < 4:  # We do this because when image is grey cv2 not add channel num to shape but keras want to see 1 as channel
    X_train.resize(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_validation.resize(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
del img_input
del steering_angle
del output
del speed
print("1st part x train shape:",X_train.shape)
print("1st part y train shape:",y_train.shape)
print("1st part laser data shape:",laser_data.shape[1])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

dataGenerator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=5,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=False,
        vertical_flip=False)
dataGenerator.fit(X_train)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 2, kernel_size = (3,3),padding = 'same',activation ='relu', input_shape = (h, w, c)))
model.add(Conv2D(filters = 4, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 24, kernel_size = (7,7),padding = 'same',activation ='relu'))

model.add(Flatten())

'''model.add(Dense(50, activation='relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(10, activation='relu'))'''

model.add(Dense(2, activation = "softmax"))

model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

In [ ]:
hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=3)
#hs = model.fit_generator(dataGenerator.flow(X_train, y_train, batch_size=64), epochs=3, validation_data=[X_validation, y_validation], steps_per_epoch=X_train.shape[0] // 64)

In [ ]:
data_start = 1000
data_stop = 2000
for part in range(1,int(len(raw_data)/1000)+1):
    if data_stop > len(raw_data):
        data_stop = len(raw_data)
    if data_stop <= data_start:
        break
    data_part = raw_data[data_start:data_stop]
    speed = data_part["speed"].values
    steering_angle = data_part["steering_angle"].values
    output = np.dstack((steering_angle, speed))[0]
    
    img_input = []
    h = 240
    w = 320
    c = 3
    for i in range(data_start+1, data_stop+1):
        img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
        img = cv2.resize(img, (w, h))
        img_input.append(img)
    img_input = np.asarray(img_input).astype(np.float32) / 255.0
    print("img input shape:",img_input.shape)
    
    X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)
    dataGenerator = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=5,
            zoom_range = 0.1, 
            width_shift_range=0.1,
            height_shift_range=0.1, 
            horizontal_flip=False,
            vertical_flip=False)
    dataGenerator.fit(X_train)
    del img_input
    del steering_angle
    del output
    del speed
    print(part+1,"th part x train shape:",X_train.shape)
    print(part+1,"th part y train shape:",y_train.shape)
    print(part+1,"th part laser data shape:",laser_data.shape[1])
    hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=3)
    #hs = model.fit_generator(dataGenerator.flow(X_train, y_train, batch_size=64), epochs=3, validation_data=[X_validation, y_validation], steps_per_epoch=X_train.shape[0] // 64)
    data_start = data_start + 1000
    data_stop = data_stop + 1000

In [ ]:
import plotly
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def trainingResultsGraph(model, modelCode):
    trace0 = go.Scatter(
        x = model.epoch,
        y = model.history['loss'],
        mode = 'lines',
        name = 'loss',
        line=dict(color='aquamarine')
    )

    trace1 = go.Scatter(
        x = model.epoch,
        y = model.history['val_loss'],
        mode = 'lines',
        name = 'val_loss',
        line=dict(color='darkred', dash='dash')
    )

    trace2 = go.Scatter(
        x = model.epoch,
        y = model.history['acc'],
        mode = 'lines',
        name = 'acc',
        line=dict(color='violet')
    )

    trace3 = go.Scatter(
        x = model.epoch,
        y = model.history['val_acc'],
        mode = 'lines',
        name = 'val_acc',
        line=dict(color='aqua', dash='dash')
    )

    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'Acc Graph',
                     method = 'update',
                     args = [{'visible': [False, False, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation accuracy'}]),
                dict(label = 'Loss Graph',
                     method = 'update',
                     args = [{'visible': [True, True, False, False]},
                             {'title': 'Trained Model'+modelCode+' training and validation loss'}]),
                dict(label = 'Both',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation values'}])
            ]),
        )
    ])

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title='Trained Model'+modelCode+' training and validation values',
                  xaxis = dict(title = 'Epochs'),
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='lossGraph')
    
trainingResultsGraph(hs, "1")

In [ ]:
try:
    import rospkg
    import rospy

    rospack = rospkg.RosPack()
    package_path = rospack.get_path('angelshark_deeplearning')
    
except ImportError:
    package_path = home_dir + "/Desktop"
    
    import os
    if not os.path.exists(package_path + "/model"):
        os.makedirs(package_path + "/model")
    
    print("Please move them to deeplearning package model directory to angelshark_deeplearning/model")
    
print("Model saved to ",str(package_path)," dir!!!")

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

In [ ]:
# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()